In [10]:
import os
import math
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchdiffeq import odeint
from tqdm import tqdm

In [11]:
#config
config = {
    'seed': 102110,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # Number of epochs.            
    'batch_size': 20, 
    'batch_time':10,
    'learning_rate': 1e-3,              
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt',  # Your model will be saved here.
    'data_size':1000
}

# 数据导入

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_size = config['data_size']
batch_time = config['batch_time']
t = torch.linspace(0., 25., data_size).to(device)#creat a t tensor  size of data_size then to device   
dataset = torch.tensor(pd.read_excel('data1.xlsx').values)
dataset = dataset.to(torch.float32)
true_y = dataset[0:1000,26].view(1000, 1, 1).to(device)

In [13]:
#class Lamda(nn.Module):
#   def forward(self, t, y):
#      return torch.mm(y, true_A)

def get_batch():
    s = torch.from_numpy(np.random.choice(np.arange(data_size - batch_time), config['batch_size'], replace=False))
    batch_y0 = true_y[s]
    batch_t = t[:batch_time]
    batch_y = torch.stack([true_y[s + i] for i in range(batch_time)], dim=0)
    return  batch_y0.to(device), batch_t.to(device), batch_y.to(device)


# 近似状态变量微分的网络
- 魔改
- 最后的输出是状态变量对时间的导数即可
- 网络的输入可以不直接使用状态变量,在求解器的前后可以加入网络结构增强

In [14]:
class ODEFunc(nn.Module):
    def __init__(self):
        super(ODEFunc, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(1, 32),
            nn.ReLU(),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Linear(16,1)
        )
    
    def forward(self, t, x):
        x = self.net(x)
        #x = x.squeeze(1)
        return x

# 求解器训练结构
- 最后的输出不能squeeze或者unsqueeze

In [ ]:
def Node_trainer():
    L_min = math.inf
    #i = list(range(config['n_epochs']))
    pbar = tqdm(list(range(config['n_epochs'])),desc='trainprocess', colour='red')
    for i in pbar:
        optimizer.zero_grad()
        batch_y0, batch_t, batch_y = get_batch()
        pred_y = odeint(func, batch_y0, batch_t).to(device)   #ode求解器（可选伴随灵敏矩阵形式）
        loss = torch.mean(torch.abs(pred_y - batch_y))
        loss.backward()
        optimizer.step()
        pbar.set_postfix({'Loss':loss.cpu().detach().numpy()})
        writer.add_scalar('Loss/Node',loss,i)
        if loss < L_min:
            L_min = loss
            torch.save(func.state_dict(), config['save_path'])
    writer.close()

func  = ODEFunc().to(device)
optimizer = optim.RMSprop(func.parameters(),lr=config['learning_rate'])
end = time.time()
writer = SummaryWriter()


Node_trainer()

In [ ]:
batch_y0, batch_t, batch_y = get_batch()
s = torch.from_numpy(np.random.choice(np.arange(data_size - batch_time), config['batch_size'], replace=False))
print(s)
batch_y0 = true_y[s]
print(batch_t.size())

tensor([436, 855, 426,  38, 357,  94,  84,  29, 180, 535, 955, 351, 684, 921,
        198,  81, 630, 764, 968, 958])
torch.Size([10])
